# **Feature Extraction Methods: Balanced Data With Annotations and Augmentation**

- *Key Features*: [MFCCs, Mel-Spectrograms, Chroma Frequencies, RMS Power]
- *Key Manipulations*: [Varying Window Sizes, Normalization, Average Pooling (Compression), Filtering]
- *Process Assistence*: [Converting them to numpy arrays now, easy label access across features]
- *Conversion*: [To numpy arrays and pkl files]


In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import time

# Libraries for audio
from IPython.display import Audio
import librosa
import librosa.display

# Training and Testing Split
from sklearn.model_selection import train_test_split

# for normalization & avgpooling features
import tensorflow as tf
# for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random
import IPython.display as ipd

In [2]:
# Variabels to be reused
path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/audio_files' 
npy_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train_audio_npy/' 
train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/train-not-annotated.csv' 
annotated_train_csv = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval-annotated.csv'
not_annotated_splt = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/trainval-split/trainval.csv'
sr = 22050

In [3]:
trainval_data = pd.read_csv(annotated_train_csv)
train_data = trainval_data[trainval_data['set'] == 'tr']
val_data = trainval_data[trainval_data['set'] == 'val']

In [4]:
train_data['species'].value_counts()

species
Dendrocopos minor            324
Ixobrychus minutus           319
Alcedo atthis                293
Acrocephalus arundinaceus    263
Motacilla flava              257
Botaurus stellaris           237
Fulica atra                  230
Charadrius alexandrinus      195
Gallinula chloropus          178
Circus aeruginosus           153
Acrocephalus melanopogon     147
Porphyrio porphyrio          142
Ardea purpurea               141
Coracias garrulus            134
Himantopus himantopus        121
Ciconia ciconia               81
Anas strepera                 68
Tachybaptus ruficollis        65
Acrocephalus scirpaceus       64
Anas platyrhynchos            32
Name: count, dtype: int64

# **Creating a class to do the extraction**

In [5]:
class Extraction:
  def __init__(self, train_df, val_df, window_size, overlap=0.5, npy_path=npy_path, sr=sr, n_mels=128, n_mfcc=20, n_chroma=12, n_cqt=84, hoplength=256, features=['mfcc'], normalize=True, avgpool=False, augment=True):
    """
    Instantiate the Extraction class to extract features.

    Parameters:
      sr (int): Sample rate of the audio files.
      n_mfccs (int): Number of MFCCs to extract.
      n_mels (int): Number of Mel bands to extract.
      n_chroma (int): Number of chroma bins to use.
      n_cqt (int): Number of CQT bins to use.
      features (list): List of features to extract.
        accepted features: 'mfcc', 'chroma', 'cqt', 'melspectrogram'.
      normalize (bool): Whether to normalize the features.
      avgpool (bool): Whether to avgpool the features.
    """
    
    self.train_df = train_df
    self.val_df = val_df
    self.npy_path = npy_path
    self.window_size = window_size
    self.overlap = overlap
    self.sr = sr
    self.n_mels = n_mels
    self.n_mfcc = n_mfcc
    self.n_chroma = n_chroma
    self.n_cqt = n_cqt
    self.hoplength = hoplength
    
     # confirm features have been specified
    assert len(features) != 0, "Must Specify At Least One Feature In The Form Of A List."
    self.features = features

    self.accepted_feature = ['mfcc', 'chroma', 'cqt', 'melspectrogram']
    for feature in self.features:
      assert feature in self.accepted_feature, f"{feature} is not an accepted feature, only 'mfcc', 'chroma', 'cqt', 'melspectrogram' are accepted features."

    self.normalize = normalize
    self.avgpool = avgpool
    self.augment = augment

    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")

    # extract train and val labels and features
    self.train_y, self.train_features, self.train_ids = self.feature_extraction(self.train_df, window_size=self.window_size)
    self.val_y, self.val_features, self.val_ids = self.feature_extraction(self.val_df, window_size=self.window_size, augment=False)

    # process the features by average pooling
    self.train_features_2D, self.val_features_2D, self.train_features_1D, self.val_features_1D = self.process_features(self.train_features, self.val_features)

  def random_augmentation(self, audio):
        """
        Apply random augmentation to the audio: pitch shift, time stretch, or add noise.
        """
        aug_type = np.random.choice(['pitch_up', 'pitch_down', 'noise', 'stretch', 'shrink'])

        if aug_type == 'pitch_up':
            # Random pitch up (between 1 and 4 semitones)
            pitch_shift = np.random.uniform(1, 4)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'pitch_down':
            # Random pitch down (between -1 and -4 semitones)
            pitch_shift = np.random.uniform(-4, -1)
            audio = librosa.effects.pitch_shift(audio, sr=self.sr, n_steps=pitch_shift)
        elif aug_type == 'noise':
            # Add random noise at a fraction of the original audio volume
            noise = self.generate_pink_noise(len(audio))
            audio = audio + 0.01 * np.random.uniform(0.2, 0.5) * np.max(audio) * noise
        elif aug_type == 'stretch':
            # Randomly stretch audio (speed up)
            stretch_factor = np.random.uniform(1.1, 1.5)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)
        elif aug_type == 'shrink':
            # Randomly shrink audio (slow down)
            stretch_factor = np.random.uniform(0.8, 0.9)
            audio = librosa.effects.time_stretch(audio, rate=stretch_factor)

        return audio

  def normalize_audio(self, audio):
    return (audio - np.min(audio)) / (np.max(audio) - np.min(audio))
  
  def generate_pink_noise(self, num_samples):
    white_noise = np.random.randn(num_samples)
    
    # Apply a filter to convert white noise into pink noise (1/f noise)
    X = np.fft.rfft(white_noise)
    S = np.arange(1, len(X) + 1)  # Frequency scaling
    pink_noise = np.fft.irfft(X / S)

    if len(pink_noise) < num_samples:
        # Pad with zeros if the length is less than num_samples
        pink_noise = np.pad(pink_noise, (0, num_samples - len(pink_noise)), mode='constant')
    elif len(pink_noise) > num_samples:
        # Trim if necessary
        pink_noise = pink_noise[:num_samples]
    
    return self.normalize_audio(pink_noise)
  
  def pad_with_noise(self, audio_data, window_length, window_samples):
    current_length = librosa.get_duration(y=audio_data, sr=self.sr)

    if current_length > window_length:
        return audio_data
    
    target_length_samples = int(window_length * sr) 
    current_length_samples = window_samples
    padding_length_samples = target_length_samples - current_length_samples

    assert target_length_samples == (current_length_samples+padding_length_samples)
    
    # Generate pink noise to pad with
    pink_noise = self.generate_pink_noise(padding_length_samples)
    padded_audio = np.concatenate([audio_data, pink_noise])
    # if len(padded_audio) < target_length_samples:
    #     padded_audio = np.append(padded_audio, self.generate_pink_noise(1))

    assert target_length_samples == len(padded_audio)
    
    return padded_audio
  
  def avg_pooling_keras(self, feature):
    # Clear the previous Keras session
    tf.keras.backend.clear_session()

    # Define the input shape based on features
    input_shape = feature.shape[1:]  # (n_mels, time_steps)

    # Create the Keras model for average pooling
    inputs = tf.keras.layers.Input(shape=input_shape)
    pooled = tf.keras.layers.GlobalAveragePooling1D()(inputs)
    pooling_model = tf.keras.models.Model(inputs=inputs, outputs=pooled)

    # Perform pooling using the model
    pooled_features = pooling_model.predict(feature)

    return pooled_features

#-------------------------Feature Extraction---------------------------------------
  def extract_mfcc(self, window):
    mfcc = librosa.feature.mfcc(y=window, sr=self.sr, n_mfcc=self.n_mfcc, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(mfcc)
    else:
      return mfcc


  def extract_chroma(self, window):
    chroma = librosa.feature.chroma_stft(y=window, sr=self.sr, n_chroma=self.n_chroma, hop_length=self.hoplength)
    if self.normalize:
      return librosa.util.normalize(chroma)
    else:
      return chroma
   

  def extract_cqt(self, window):
    cqt = librosa.cqt(y=window, sr=sr, hop_length=self.hoplength, n_bins=self.n_cqt)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)
    return cqt_db

  def extract_melspectrogram(self, window):
    mel = librosa.feature.melspectrogram(y=window, sr=self.sr, n_mels=self.n_mels, hop_length=self.hoplength)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if self.normalize:
      return librosa.util.normalize(mel_db)
    else:
      return mel_db
    
  def avgpooling(self, train_X, val_X, n_time, n_features):
    """
    Average pooling the train and val features.

    Parameters:
      train_X (npy): Training feature array of shape (batch_size, n_features, n_time)
      val_X (npy): Validation feature array of shape (batch_size, n_features, n_time)
      n_time (int): Time axis
      n_features (int): Feature axis

    Returns:
      train_X (npy): Avgpooled training feature array of shape (batch_size, n_features)
      val_X (npy): Avgpooled validation feature array of shape (batch_size, n_features)
    """
    # Clear the Keras session
    tf.keras.backend.clear_session()
    
    # Create the Keras input layer with shape (n_features, n_time)
    input_layer = tf.keras.layers.Input(shape=(n_features, n_time))
    
    # Apply average pooling over the time axis (axis=-1) to reduce n_time
    avg_pool = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1))(input_layer)
    
    # Build the model
    pooling_model = tf.keras.models.Model(inputs=input_layer, outputs=avg_pool)

    # Use the model to apply average pooling on the training and validation features
    train_X = pooling_model.predict(train_X)
    val_X = pooling_model.predict(val_X)

    return train_X, val_X

    
  def process_features(self, train_features_dict, val_features_dict):
    train_copy = train_features_dict.copy()
    val_copy = val_features_dict.copy()
    for each in train_copy.keys():
      
      if each == 'mfcc':
        n_features=self.n_mfcc
      elif each == 'chroma':
        n_features=self.n_chroma
      elif each == 'cqt':
        n_features=self.n_cqt
      elif each == 'melspectrogram':
        n_features=self.n_mels
      
      train_feature = train_copy[each]
      val_feature = val_copy[each]

      if self.avgpool:
        train_copy[each], val_copy[each] = self.avgpooling(train_feature, val_feature, n_time=train_feature.shape[2], n_features=n_features)
      else:
        train_copy[each], val_copy[each] = train_copy[each], val_copy[each]
    
    return train_features_dict, val_features_dict, train_copy, val_copy
      

  def feature_extraction(self, dataframe, window_size, augment=True):
    y = [] # To hold the labels
    ids = []
    features_dict = {item: [] for item in self.features} # Create a key for each feature listed
    print(f"Number of rows in dataframe: {len(dataframe)}")
    for _, row in tqdm(dataframe.iterrows(), desc="Processing data", total=len(dataframe)):
          label = row['species']
          file_path = os.path.join(self.npy_path, row['filename_npy'])
          id = row['audio_name']
          start = row['start']
          end = row['end']

          # print(f"Processing file: {file_path}")

          try:
            
              audio = np.load(file_path)
          except FileNotFoundError:
              print(f"File not found: {file_path}")
              continue


          start = int(start * sr)
          end = int(end * sr)+512



          if end > len(audio):
             end = len(audio)

          sample = audio[start:end]

          if len(sample) < 512:
                continue

          sample = self.normalize_audio(sample)
          if augment:
                sample = self.random_augmentation(sample)

          sample = self.pad_with_noise(sample, window_length=self.window_size, window_samples=len(sample))
          # print(len(sample))

          window_samples = int(window_size * self.sr)
          hop_samples = int(window_samples * (1 - self.overlap))  # For overlapping

          # Break the audio into windows with the specified overlap
          audio_windows = librosa.util.frame(sample, frame_length=window_samples, hop_length=hop_samples).T
          
          
          # display(label)
          
          for _, window in enumerate(audio_windows):
              
              y.append(label)
              ids.append(id)

              if len(window) < window_samples:
                  if len(window) < 512*2:
                     continue
                  else:
                      window = self.pad_with_noise(window, window_length=window_size)
              
              # Feature Extraction FR --------------------------------------------------------------------
              # dynatically call the extract_x function to extract the listed features
              for feature in self.features:
                extract = f"extract_{feature}"
                if hasattr(self, extract) and callable(func := getattr(self, extract)):
                  features_dict[feature].append(func(window))

          # cast lists to np arrays
    for each in features_dict.keys():
              features_dict[each] = np.array(features_dict[each])

    y = np.array(y)
    ids = np.array(ids)

          # If not using average pooling, return resized features
    return y, features_dict, ids

# **Trying to balance the training set**

In [6]:
# Step 1: Identify the majority class count
species_counts = train_data['species'].value_counts()
majority_class_count = species_counts.max()

# Step 2: Define the target for oversampling (0.7 times the majority class count)
target_count = int(majority_class_count * 0.7)

# Step 3: Separate majority and minority classes
df_majority = train_data[train_data['species'].map(species_counts) == majority_class_count]

# Step 4: Oversample the minority classes to the target count
oversampled_data = pd.DataFrame()

for species, count in species_counts.items():
    species_df = train_data[train_data['species'] == species]
    if count < target_count:
        species_upsampled = resample(species_df, 
                                     replace=True,     # Allow oversampling
                                     n_samples=target_count,  # Target count for each class
                                     random_state=42)  # For reproducibility
        oversampled_data = pd.concat([oversampled_data, species_upsampled])
    else:
        oversampled_data = pd.concat([oversampled_data, species_df])

# Combine with majority class data
train_data_balanced = pd.concat([oversampled_data])

# Check the new class distribution
print(train_data_balanced['species'].value_counts())

species
Dendrocopos minor            324
Ixobrychus minutus           319
Alcedo atthis                293
Acrocephalus arundinaceus    263
Motacilla flava              257
Botaurus stellaris           237
Fulica atra                  230
Coracias garrulus            226
Acrocephalus scirpaceus      226
Tachybaptus ruficollis       226
Anas strepera                226
Ciconia ciconia              226
Himantopus himantopus        226
Acrocephalus melanopogon     226
Ardea purpurea               226
Porphyrio porphyrio          226
Circus aeruginosus           226
Gallinula chloropus          226
Charadrius alexandrinus      226
Anas platyrhynchos           226
Name: count, dtype: int64


## **Window Size = 3s**

In [7]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [8]:
features = Extraction(train_data_balanced,
                      val_data,
                      window_size=3,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (4861, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 4861


Processing data:   0%|          | 0/4861 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data:  19%|█▉        | 923/4861 [01:16<05:45, 11.40it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1959
  warnings.warn(
Processing data:  43%|████▎     | 2071/4861 [03:15<05:51,  7.95it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1927
  warnings.warn(
Processing data:  47%|████▋     | 2288/4861 [03:42<05:05,  8.43it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1986
  

Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [01:30<00:00,  9.17it/s]



288/288 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
288/288 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step


In [9]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(9197,)

(1350,)

In [10]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(9197, 128)

'mfcc'

(9197, 20)

'chroma'

(9197, 12)

'cqt'

(9197, 84)

In [11]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(1350, 128)

'mfcc'

(1350, 20)

'chroma'

(1350, 12)

'cqt'

(1350, 84)

In [12]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(9197, 128, 259)

'mfcc'

(9197, 20, 259)

'chroma'

(9197, 12, 259)

'cqt'

(9197, 84, 259)

In [13]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(1350, 128, 259)

'mfcc'

(1350, 20, 259)

'chroma'

(1350, 12, 259)

'cqt'

(1350, 84, 259)

In [14]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [15]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [12 12 12 ...  4  4  4]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [16]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [17]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.08778238, -0.42542818, -0.45237106, ..., -0.9883016 ,
          -0.99088913, -0.9917844 ],
         [-0.08818693, -0.4527624 , -0.48460808, ..., -0.9879749 ,
          -0.9912214 , -0.99346757],
         [-0.06666847, -0.4327347 , -0.45371366, ..., -0.98730004,
          -0.9916132 , -0.9961998 ],
         ...,
         [-0.01765106, -0.9769646 , -0.97906095, ..., -0.86725354,
          -0.8884754 , -0.9976066 ],
         [-0.01777176, -0.7929504 , -0.7817982 , ..., -0.98777676,
          -0.9978452 , -1.        ],
         [-0.03171015, -0.9459309 , -0.8921383 , ..., -0.9837095 ,
          -0.9915168 , -0.9999268 ]], dtype=float32),
  'mfcc': array([[-0.9975469 ,  0.5757494 , -0.13481264, ...,  0.03258222,
           0.01895186,  0.02151072],
         [-0.99573815,  0.5883738 , -0.2239488 , ...,  0.03442121,
           0.02149916,  0.03377577],
         [-0.9786744 ,  0.7425647 , -0.30753863, ...,  0.03714854,
           0.02035247,  0.02320485]

In [18]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-0.085169  , -0.06403001, -0.06252546, ..., -0.18273787,
           -0.18892042, -0.20181585],
          [-0.25508188, -0.27114728, -0.32115506, ..., -0.38445121,
           -0.39873584, -0.3933129 ],
          [-0.32371444, -0.33958841, -0.38829561, ..., -0.45105063,
           -0.444728  , -0.43501664],
          ...,
          [-0.9979125 , -0.99782803, -0.99764849, ..., -0.99674981,
           -1.        , -0.99783832],
          [-0.99790549, -0.99786567, -0.99781859, ..., -0.99581807,
           -0.99702104, -0.99504838],
          [-1.        , -1.        , -1.        , ..., -1.        ,
           -0.99911522, -0.99625083]],
  
         [[-0.07655078, -0.05783229, -0.05719943, ..., -0.06720283,
           -0.05434324, -0.06117152],
          [-0.22979912, -0.2450997 , -0.29211519, ..., -0.29226506,
           -0.26610162, -0.24624838],
          [-0.28998886, -0.30498022, -0.35125536, ..., -0.38577562,
           -0.34912624, -0.32429723],


### Save the merged dictionary to a pkl

In [19]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Augmented/AveragePooled/split_features_3s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [20]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Augmented/NotAveragePooled/split_features_3s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file

## **Window Size = 1s**

In [21]:
features_list = ['melspectrogram', 'mfcc', 'chroma', 'cqt']

In [22]:
features = Extraction(train_data_balanced,
                      val_data,
                      window_size=1,
                      features=features_list,
                      avgpool=True
                      )

Train DataFrame shape: (4861, 9)
Validation DataFrame shape: (834, 9)
Number of rows in dataframe: 4861


Processing data:   0%|          | 0/4861 [00:00<?, ?it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
Processing data:  19%|█▉        | 923/4861 [00:56<05:04, 12.95it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1959
  warnings.warn(
Processing data:  43%|████▎     | 2070/4861 [03:20<03:03, 15.18it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1927
  warnings.warn(
Processing data:  45%|████▌     | 2190/4861 [03:28<03:13, 13.79it/s]c:\Users\thato\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1928
  

Number of rows in dataframe: 834


Processing data: 100%|██████████| 834/834 [02:09<00:00,  6.44it/s]


787/787 ━━━━━━━━━━━━━━━━━━━━ 1s 701us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step
787/787 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step
787/787 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 435us/step
787/787 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step


In [23]:
train_y = features.train_y
val_y = features.val_y

display(train_y.shape)
display(val_y.shape)

(25159,)

(3318,)

In [24]:
# Avgpooled Features
train_features_1D = features.train_features_1D
for key in train_features_1D.keys():
  display(key)
  display(train_features_1D[key].shape)

'melspectrogram'

(25159, 128)

'mfcc'

(25159, 20)

'chroma'

(25159, 12)

'cqt'

(25159, 84)

In [25]:
# Avgpooled Features
val_features_1D = features.val_features_1D
for key in val_features_1D.keys():
  display(key)
  display(val_features_1D[key].shape)

'melspectrogram'

(3318, 128)

'mfcc'

(3318, 20)

'chroma'

(3318, 12)

'cqt'

(3318, 84)

In [26]:
# Not avgpooled Features
train_features_2D = features.train_features_2D
for key in train_features_2D.keys():
  display(key)
  display(train_features_2D[key].shape)

'melspectrogram'

(25159, 128, 87)

'mfcc'

(25159, 20, 87)

'chroma'

(25159, 12, 87)

'cqt'

(25159, 84, 87)

In [27]:
# Not avgpooled Features
val_features_2D = features.val_features_2D
for key in val_features_2D.keys():
  display(key)
  display(val_features_2D[key].shape)

'melspectrogram'

(3318, 128, 87)

'mfcc'

(3318, 20, 87)

'chroma'

(3318, 12, 87)

'cqt'

(3318, 84, 87)

In [28]:
train_ids = features.train_ids
val_ids = features.val_ids

### Encode Classes

In [29]:
label_encoder = LabelEncoder().fit(train_y)
train_y_encoded = label_encoder.transform(train_y)
val_y_encoded = label_encoder.transform(val_y)

classes = list(label_encoder.inverse_transform([0, 1, 2]))
print("Encoded classes for [0, 1, 2]:", classes)
print("Encoded training labels:", train_y_encoded)
print("Encoded validation labels:", val_y_encoded)

Encoded classes for [0, 1, 2]: ['Acrocephalus arundinaceus', 'Acrocephalus melanopogon', 'Acrocephalus scirpaceus']
Encoded training labels: [12 12 12 ...  4  4  4]
Encoded validation labels: [14 14 14 ...  4  4  4]


In [30]:
# Avg Pooled

train_features_1D['label'] = train_y_encoded
val_features_1D['label'] = val_y_encoded

train_features_1D['id'] = train_ids
val_features_1D['id'] = val_ids

# Not Avg Pooled

train_features_2D['label'] = train_y_encoded
val_features_2D['label'] = val_y_encoded

train_features_2D['id'] = train_ids
val_features_2D['id'] = val_ids

In [31]:
merged_dict_1D = {'train': train_features_1D, 'val': val_features_1D}
merged_dict_1D

{'train': {'melspectrogram': array([[-0.04504155, -0.5129219 , -0.47196412, ..., -0.9712567 ,
          -0.99281067, -0.99828345],
         [-0.01699185, -0.51559293, -0.53285223, ..., -0.9912033 ,
          -0.99977577, -1.        ],
         [-0.01728415, -0.5161555 , -0.49409568, ..., -0.9978849 ,
          -0.99978524, -0.9999946 ],
         ...,
         [-0.0171945 , -0.8915984 , -0.8546513 , ..., -0.99450314,
          -0.9996007 , -1.        ],
         [-0.01721412, -0.9050265 , -0.8571305 , ..., -0.99341494,
          -0.99945045, -0.99988985],
         [-0.01719472, -0.9099785 , -0.8790836 , ..., -0.99427086,
          -0.9995546 , -1.        ]], dtype=float32),
  'mfcc': array([[-0.965824  ,  0.7495787 , -0.56103367, ...,  0.01619458,
           0.04867846,  0.05167317],
         [-0.9794857 ,  0.8005483 , -0.5696797 , ...,  0.05881221,
           0.02247417,  0.03910745],
         [-0.9795871 ,  0.84554815, -0.5713744 , ...,  0.04461651,
           0.01746377,  0.03254361]

In [32]:
merged_dict_2D = {'train': train_features_2D, 'val': val_features_2D}
merged_dict_2D

{'train': {'melspectrogram': array([[[-3.67881385e-02, -1.84207382e-02, -2.13169485e-02, ...,
           -3.30114481e-02, -4.38906960e-02, -4.98871926e-02],
          [-1.94555980e-01, -2.12092587e-01, -2.64288687e-01, ...,
           -1.90025605e-01, -1.55930098e-01, -1.45294745e-01],
          [-2.51263802e-01, -2.65686724e-01, -3.12424335e-01, ...,
           -2.59840115e-01, -2.19372969e-01, -2.05539240e-01],
          ...,
          [-9.07598412e-01, -9.08638426e-01, -9.12845269e-01, ...,
           -9.70791936e-01, -9.62711216e-01, -9.53373095e-01],
          [-9.21042425e-01, -9.21481495e-01, -9.24874317e-01, ...,
           -1.00000000e+00, -9.75673073e-01, -9.62109094e-01],
          [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
           -9.99230228e-01, -9.94950480e-01, -9.86188374e-01]],
  
         [[-1.76479556e-02, -1.33084832e-04, -7.89614767e-03, ...,
           -9.56111681e-03,  3.35906343e-08, -1.28647378e-02],
          [-1.94707453e-01, -2.13216081e-01,

### Save the merged dictionary to a pkl

In [33]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Augmented/AveragePooled/split_features_1s_all_1D.pkl', 'wb') as file:
  pickle.dump(merged_dict_1D, file)
del file

In [34]:
# Avrg Pooled
with open('C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-Of-Bird-Classification-Project/feature-extraction/Annotated/Augmented/NotAveragePooled/split_features_1s_all_2D.pkl', 'wb') as file:
  pickle.dump(merged_dict_2D, file)
del file